# Chain of Thought: Solution variants

Illustrates using Chain of Thought to fix some of the problematic queries

In [1]:
#%pip install --upgrade --quiet pydantic-ai-slim[anthropic,openai]

In [1]:
GEMINI="gemini-2.0-flash"
OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please specify the OPENAI_API_KEY access token in keys.env file"

In [2]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio
nest_asyncio.apply()

In [3]:
def zero_shot(model_id, prompt: str) -> str:
    from pydantic_ai import Agent
    agent = Agent(model_id)
    result = agent.run_sync(prompt)
    return (result.data)

## Zero-shot CoT

Simply saying "think about it step-by-step"

In [8]:
print(zero_shot(CLAUDE, """
If I have a pipe of diameter 25cm and a hose that is 100m long,
what's the flow-rate of Texas Sweet with a 7 bar pressure differential?
Think about it step-by-step.
"""))

# Flow Rate Calculation for Texas Sweet Crude Oil

To determine the flow rate of Texas Sweet crude oil through a pipe, I'll need to apply fluid dynamics principles, specifically the Darcy-Weisbach equation or Hagen-Poiseuille law.

## Given information:
- Pipe diameter (D) = 25 cm = 0.25 m
- Pipe length (L) = 100 m
- Pressure differential (ΔP) = 7 bar = 700,000 Pa
- Fluid: Texas Sweet crude oil

## Step 1: Determine the properties of Texas Sweet crude oil
Texas Sweet crude oil typically has these properties:
- Density (ρ): approximately 850 kg/m³
- Dynamic viscosity (μ): approximately 0.005-0.010 Pa·s (I'll use 0.008 Pa·s)

## Step 2: Calculate flow rate using the Hagen-Poiseuille equation
For laminar flow in a circular pipe:
Q = (π × D⁴ × ΔP) / (128 × μ × L)

Where:
- Q = volumetric flow rate (m³/s)
- π = 3.14159
- D = pipe diameter (m)
- ΔP = pressure differential (Pa)
- μ = dynamic viscosity (Pa·s)
- L = pipe length (m)

## Step 3: Substitute values and calculate
Q = (π × (0.25 m)⁴ 

## Few-shot CoT

Providing an example or two.

In [10]:
print(zero_shot(GEMINI, """
You are allowed to check in 50kg if your final destination is the United States or if you have special needs.
Otherwise, you are allowed only 40kg. 

Example 1:
Q: What's the baggage allowance for itinerary CDG-ATL-SEA?
A: The final destination, SEA, is in the United States. So, baggage allowance is 50kg.

Example 2:
Q: What's the baggage allowance for itinerary CDG-LHR-NBO?
A: The final destination, NBO, is in Kenya. So, baggage allowance is 40kg.

Q: What is the baggage allowance for the following itineary: SIN-DFW-YYZ?
"""))

A: The final destination, YYZ, is in Canada. So, baggage allowance is 40kg.



## Where CoT won't help

CoT doesn't help if the problem is a data gap.
To solve this question, you need to give the model access to a map.
You also need a model capable of interpreting a map.
Currently (April 2025), Claude Sonnet can do this, but GPT-4o-mini cannot.

In [17]:
def answer_based_on_map(model_id, prompt: str) -> str:
    from pydantic_ai import Agent, ImageUrl
    agent = Agent(model_id)
    result = agent.run_sync([prompt, 
                             ImageUrl(url="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/India_roadway_map.svg/1639px-India_roadway_map.svg.png")
                            ])
    return (result.data)

In [18]:
print(answer_based_on_map(OPENAI,"""
If I drive 300km west of Hyderabad, where will I land up?
Think about it step-by-step.
"""))

If you drive 300 km west of Hyderabad, you can expect to land in or around a few notable areas in the state of Maharashtra. Here's a step-by-step breakdown:

1. **Starting Point**: You begin in Hyderabad, which is located in the southern part of India, in the state of Telangana.
  
2. **Direction**: Driving west means you'll be heading toward Maharashtra, which is directly west of Telangana.

3. **Estimated Distance**: A distance of 300 km takes you through various terrain and could put you near several towns or cities in Maharashtra.

4. **Potential Destinations**:
   - **Aurangabad**: A significant city known for its historical sites, including the Ajanta and Ellora Caves. It is about 300 km from Hyderabad.
   - **Nanded**: Another city that is approximately within this distance and is known for its Gurudwara and historical significance in Sikhism.

5. **Route Options**: Depending on the specific route taken (e.g., via NH65), you might pass through smaller towns as well, like Zaheera

In [20]:
print(answer_based_on_map(CLAUDE,"""
If I drive 300km west of Hyderabad, where will I land up?
Think about it step-by-step.
"""))

Looking at the India National Highway Network Map, I'll determine where you would end up if you drive 300km west of Hyderabad.

Step 1: Locate Hyderabad on the map.
I can see Hyderabad in southern India, in the state of Telangana.

Step 2: Determine what's 300km west of Hyderabad.
Moving west from Hyderabad for approximately 300km would take you toward the western part of Maharashtra state. Based on the map scale and the highway network, this would put you somewhere near or in the vicinity of:

- You would likely be close to areas between Solapur and Mumbai in Maharashtra
- You might be in the eastern parts of Maharashtra, possibly near cities like Nanded or Latur
- You could potentially be approaching the western Maharashtra region

Step 3: Conclusion
If you drive 300km west of Hyderabad, you would most likely end up in western Maharashtra, possibly near Solapur or somewhere between Solapur and the Maharashtra-Karnataka border region. The exact location would depend on the specific ro